# Sesión 06: ANOVA de un factor y Prueba Kruskal-Wallis

El análisis de varianza (ANOVA) describe a un conjunto técnicas estadísticas que se utilizan para comparar la media entre dos o más grupos. La hipótesis nula es que todas las medias son iguales mientras que la hipótesis alternativa plantea que al menos una es diferente.

El análisis de varianza de un factor implica una variable dependiente continua y una variable independiente (llamada factor) que tiene cierto número de niveles. Los diferentes niveles de la variable independiente corresponden a diferente grupos o condiciones. Cuando existen dos o más factores el análisis se le llama análisis de varianza de n factores. Si el conjunto de variables independientes incluye variables cuantitativas (covariables), a la técnica se le llama análisis de covarianza (ANCOVA). Si hay dos o más variables dependientes se le conoce como MANOVA.

El análisis de varianza recibe su nombre debido a que compara la variabilidad entre grupos con la variabilidad dentro de los grupos. Se calcula un valor F que representa la variabilidad entre los grupos dividida entre la variabilidad dentro de los grupos. Un valor grande de la F indica que hay más variabilidad entre los grupos (causados por la variable independiente) que la que hay dentro de cada grupo (atribuida al azar).

Un valor significativo en la F implica que se rechaza la hipótesis nula, es decir, que al menos una media es diferente. Para determinar qué grupo es diferente se deben llevar a cabo pruebas post-hoc o realizar comparaciones específicas. 

El ANOVA de un factor requiere el cumplimiento de varios supuestos: las muestras son independientes, la variable dependiente está normalmente distribuida en cada grupo, y las varianzas en cada grupo son iguales. Debido a que la ANOVA se considera como una prueba robusta, variaciones en el cumplimiento de los supuestos de normalidad y homogeneidad de varianzas no afectan de manera importante los resultados si el tamaño de las muestras es igual. 

**Caso de uso**
Una cadena de comida rápida planea agregar un nuevo producto a su menú, pero están indecisos entre tres posibles campañas de mercadotecnica. Como un experimento, el producto fue introducido en varias ubicaciones seleccionadas aleatoriamente utilizando diferentes campañas. Se registraron las ventas del nuevo producto por las primeras cuatro semanas. El archivo "marketing.csv" contiene las siguientes variables:  
- MarketID: identificador del mercado
- MarketSize: tamaño del mercado de acuerdo a las ventas
- LocationID: identificador de la ubicación de la tienda
- AgeOfStore: antigüedad de la tienda en años
- Promotion: promoción (1 de 3) que fue probada
- week: semana en que se llevó a cabo la promoción.
- SalesInThousands: ventas para una ubicación específica (LocationID), promoción (Promotion) y semana (week).

Para comparar las ventas de acuerdo a las diferentes promociones, utilizaremos una ANOVA  
Hipótesis nula: todas las medias son iguales.  
Hipótesis alternativa: al menos un par es diferente.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/marketing.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MarketID          548 non-null    int64  
 1   MarketSize        548 non-null    object 
 2   LocationID        548 non-null    int64  
 3   AgeOfStore        548 non-null    int64  
 4   Promotion         548 non-null    int64  
 5   week              548 non-null    int64  
 6   SalesInThousands  548 non-null    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 30.1+ KB


In [4]:
df['Promotion'].value_counts()

Promotion
3    188
2    188
1    172
Name: count, dtype: int64

In [5]:
df.sample(4)

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
285,6,Medium,507,5,2,2,17.34
301,6,Medium,511,24,3,2,39.41
73,2,Small,106,5,3,2,62.19
271,6,Medium,503,2,2,4,37.20


## Supuesto de igualdad de varianzas

In [7]:
from scipy.stats import levene

# Filtramos los datos para obtener los grupos
promocion1 = df[df['Promotion'] == 1]['SalesInThousands'] 
promocion2 = df[df['Promotion'] == 2]['SalesInThousands'] 
promocion3 = df[df['Promotion'] == 3]['SalesInThousands'] 

# Realizar la prueba
statistic, p_value = levene(promocion1, promocion2, promocion3)
print("Estadístico de prueba de Levene:", statistic)
print("Valor p:", p_value)

Estadístico de prueba de Levene: 1.2696787527507418
Valor p: 0.2817514565692118


In [8]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.


## Supuesto de normalidad

In [9]:
from scipy import stats

def test_normality(data):
    mean = data.mean()
    std = data.std()
    return stats.kstest(data, 'norm', args=(mean, std))

# Realizar la prueba K-S para cada grupo de promoción
for promo in df['Promotion'].unique():
    sales_data = df[df['Promotion'] == promo]['SalesInThousands']
    stat, p_value = test_normality(sales_data)
    print(f'Promoción {promo}: Estadístico K-S = {stat}, Valor p = {p_value}')

Promoción 3: Estadístico K-S = 0.14589472052806318, Valor p = 0.0005892335857854408
Promoción 2: Estadístico K-S = 0.14252750909757217, Valor p = 0.0008534307515990372
Promoción 1: Estadístico K-S = 0.12268074514406724, Valor p = 0.010284902762809082


Sin embargo, varios estudios muestran que la prueba ANOVA es robusta ante violaciones de la normalidad si:
- Los grupos tienen un tamaño similar
- Hay por lo menos 40 observaciones en cada grupo.

## ANOVA

In [10]:
from scipy.stats import f_oneway

# Filtramos los datos para obtener los grupos
promocion1 = df[df['Promotion'] == 1]['SalesInThousands'] 
promocion2 = df[df['Promotion'] == 2]['SalesInThousands'] 
promocion3 = df[df['Promotion'] == 3]['SalesInThousands'] 


# Aplicar la prueba ANOVA
statistic, p_value = f_oneway(promocion1, promocion2, promocion3)

# Imprimir los resultados
print("Estadístico de prueba de ANOVA:", statistic)
print("Valor p:", p_value)

Estadístico de prueba de ANOVA: 21.953485793080677
Valor p: 6.765849261408714e-10


In [11]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay al menos una diferencia significativa entre los grupos.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas entre los grupos.")

Se rechaza la hipótesis nula: hay al menos una diferencia significativa entre los grupos.


La tabla ANOVA no indica qué grupo es diferente al resto, sin embargo, las pruebas post-hoc son útiles para detectar qué grupo es diferente al resto. 
- LSD/DMS (Diferencia menos significativa): Es el equivalente a múltiples pruebas t, no se hacen correcciones y los resultados no son precisos.
- Bonferroni: Corrige el nivel de significancia dividiéndolo entre el número de grupos. Es preferible cuando son pocas comparaciones. 
- Tukey: Preferible cuando son muchas comparaciones. Es deseable que el tamaño de cada grupo sea igual. 
- REGWQ (Ryan-Einot-Gabriel-Welsh): Recomendable, pero se debe evitar cuando las muestras son de diferente tamaño.
- Dunnett: Es apropiada cuando se desea comparar con un grupo de control.
- Gabriel: Apropiada cuando el tamaño de las muestras es ligeramente diferente.
- GT2 de Hochberg: Apropiadas cuando el tamaño de las muestras es muy diferente
- Games- Howell: Recomendable cuando las varianzas son diferentes.

La librería statsmodel permite realizar la prueba de Tukey, Bonferroni y Dunnet

In [12]:
from statsmodels.stats.multicomp import MultiComparison

comparison = MultiComparison(df["SalesInThousands"], df["Promotion"])
tukey = comparison.tukeyhsd(0.05)
print(tukey.summary())

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     1      2 -10.7696    0.0 -14.7738 -6.7654   True
     1      3  -2.7345 0.2444  -6.7388  1.2697  False
     2      3   8.0351    0.0   4.1208 11.9493   True
-----------------------------------------------------


Ejemplo de redacción de conclusiones: 

>Se realizó un ANOVA de un factor para analizar cómo ________ influye en _________. Los resultados muestran que _________tiene un efecto significativo en __________, F(_,_) = ___, p = ___. Las comparaciones post-hoc utilizando el método de Tukey HSD indican que la media de ______ es significativamente diferente a la media de _________ y _______

## Prueba Kruskal-Wallis

In [13]:
from scipy.stats import kruskal

# Filtramos los datos para obtener los grupos
promocion1 = df[df['Promotion'] == 1]['SalesInThousands'] 
promocion2 = df[df['Promotion'] == 2]['SalesInThousands'] 
promocion3 = df[df['Promotion'] == 3]['SalesInThousands'] 

# Aplicar la prueba Kruskal-Wallis
statistic, p_value = kruskal(promocion1, promocion2, promocion3)

# Imprimir los resultados
print("Estadístico de prueba de Kruskal-Wallis:", statistic)
print("Valor p:", p_value)


Estadístico de prueba de Kruskal-Wallis: 53.29475169322799
Valor p: 2.6741866266697816e-12


In [ ]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas entre los grupos.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas entre los grupos.")